# Marketing Mix Modeling in Python

![banner_marketing.png](https://github.com/Chesar832/Marketing_modeling_mix/blob/main/AdvancedVersion/img/banner_marketing.png?raw=true)

*El presente tiene por objetivo analizar la data proporcionada por Kaggle para el área de Marketing; se busca encontrar "**Qué inversión publicitaria impulsa en mayor porporción a las ventas**"*

## 📤 Librerias

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_profiling import ProfileReport
import altair as alt

## 💾 Carga de datos

In [2]:
mrk_data = pd.read_csv('data/new_dataset.csv', delimiter=';')

In [3]:
mrk_data.sample(5)

,WeekDate,Influencer,TV,Radio,Social Media,Sales
570,19/09/2020,Micro,421,142.202630,25.976138,1497.541283
6,6/01/2018,Micro,355,113.528701,24.262875,1269.989343
364,28/09/2019,Macro,252,77.420116,11.135351,908.435843
573,26/09/2020,Mega,418,150.860655,28.777353,1481.532251
349,31/08/2019,Mega,379,114.706799,29.513674,1347.478712


# 📊 EDA

El [dataset](https://docs.google.com/spreadsheets/d/1d_XEzDSvhkfWHeNj3Ux5Y5KerjYGEDDB/edit?usp=sharing&ouid=100459174823708459699&rtpof=true&sd=true) con el que se va a trabajar contiene las siguientes variables:

|  **VARIABLE**  |                                          **DESCRIPCIÓN**                                      |
| :---           |                                                                                               | 
| WeekDate       | Fecha de la semana en la que se culminó la promoción                                          |
| TV             | Presupuesto de promoción televisiva *(en millones)*                                           |
| Social Media   | Presupuesto de promoción de redes sociales *(en millones)*                                    |
| Radio          | Presupuesto de promoción radiofónica *(en millones)*                                          |
| Influencer     | Tipo de influencer con el que colabora en la promoción (Mega, Macro, Nano o Micro influencer) |
| Sales          | Ventas obtenidas *(en millones)*                                                              |

In [4]:
mrk_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 655 entries, 0 to 654
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   WeekDate      655 non-null    object 
 1   Influencer    655 non-null    object 
 2   TV            655 non-null    int64  
 3   Radio         655 non-null    float64
 4   Social Media  655 non-null    float64
 5   Sales         655 non-null    float64
dtypes: float64(3), int64(1), object(2)
memory usage: 30.8+ KB


### Corrigiendo formatos

In [5]:
mrk_data['WeekDate'] = pd.to_datetime(mrk_data['WeekDate'])

### Generando el reporte general

In [6]:
report = ProfileReport(mrk_data, title="Marketing modeling mix", explorative=True)
report.to_file("marketing-report.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

Del reporte generado:
- Ninguna variable contiene valores faltantes.
- No existen filas que contengan valores duplicados.
- Las variables TV e influencer están fuertemente correlacionadas(esto puede ser ocasionado por la naturaleza publicitaria de los influencers en los medios audiovisuales).
- La distribución de los tipos de influencers es casi simétrica.
- La distribución de los presupuestos de publicidad televisiva, radiofónica, por redes sociales y las ventas es casi simétrica.
- Los presupuestos para marketing televisivo están más fuertemente relacionados a las ventas generadas que las otras variables.

### Análisis bivariante

#### Análisis Bivariante Con respecto a las Ventas

In [7]:
alt.Chart(mrk_data).mark_circle(size=40,stroke='#000000', strokeWidth=0.2).encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
    color='Influencer:N',
    tooltip=[alt.Tooltip('Sales', title='Ventas generadas',format='~s')]
).properties(
    width=200,
    height=230
).repeat(
    row=['Sales'],
    column=['Radio', 'TV', 'Social Media']
).interactive()

alt.RepeatChart(...)

→ Los 3 features de Radio, TV y Social Media están relacionados de manera lineal. </br>
→ Los presupuestos destinados a publicidad televisiva están más fuertemente relacionados a las ventas generadas en esa campaña de publicidad.</br>
→ Las 3 variables tienen una relacón de tipo lineal con las ventas obtenidas.

In [8]:
table_temp_01 = pd.DataFrame(mrk_data.groupby('Influencer')[['Radio','TV','Social Media','Sales']].sum())
table_temp_01 = table_temp_01.reset_index()
table_temp_01

,Influencer,Radio,TV,Social Media,Sales
0,Macro,20538.879787,61488,3770.103797,219282.846473
1,Mega,20932.175593,61846,3899.961651,220326.277957
2,Micro,20983.456725,61979,3774.083487,220964.077721
3,Nano,20501.993612,61340,3733.037354,218229.303096


In [9]:
bars_01 = alt.Chart(table_temp_01).mark_bar().encode(
    x=alt.X('Influencer:O', title='', axis=None),
    y=alt.Y('Sales:Q',title='Ventas en Millones', stack=None, axis=alt.Axis(titlePadding=20, titleAnchor='middle' ,labelFontSize=12, grid=False)),
    color="Influencer:N",
    tooltip = [alt.Tooltip('Influencer:O',title= 'Tipo de influencer'),
               alt.Tooltip('Sales:Q', title = 'Ventas en Millones de dólares',format='~s')],
)

In [10]:
labels_01 = bars_01.mark_text(size=15,dy = -10).encode(
    text = alt.Text('Sales:Q', format = "~s"),
)

In [11]:
chart_01 = bars_01 + labels_01

In [12]:
chart_01.properties(
    title = 'Ventas totales en millones de dólares por tipo de influencer',
    width = 400,
    height = 300
).configure_title(
    fontSize = 14,
    anchor = 'middle'
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)

→ **No existe una diferencia representativa determinada por el tipo de influencer** con el que se realiza la estrategia publicitaria.

In [13]:
table_temp_02= pd.DataFrame(mrk_data, columns=['WeekDate','Sales'])
table_temp_02['Month'] = table_temp_02.apply(lambda x: x['WeekDate'].month , axis=1)
table_temp_02 = table_temp_02.sort_values('Month')
table_temp_02['Month'] = table_temp_02.apply(lambda x: x['WeekDate'].month_name() , axis=1)

In [14]:
dict_months = { 
    'January':'Enero',
    'February':'Febrero',
    'March':'Marzo',
    'April':'Abril',
    'May':'Mayo',
    'June':'Junio',
    'July':'Julio',
    'August':'Agosto',
    'September':'Septiembre',
    'October':'Octubre',
    'November':'Noviembre',
    'December':'Diciembre'
}

In [15]:
table_temp_02['year'] = table_temp_02.apply(lambda x: x['WeekDate'].year , axis=1)
table_temp_02 = table_temp_02.replace({'Month':dict_months})

In [16]:
table_temp_02 = table_temp_02.pivot_table(values='Sales',index='Month',columns='year', fill_value=0, sort=False).reset_index()
table_temp_02 = table_temp_02.melt(id_vars=['Month'],value_vars=[2017,2018,2019,2020,2021], var_name='NameYear', value_name='Sales')

In [17]:
alt.Chart(table_temp_02).mark_line(point = True, size=3).encode(
    x = alt.X("Month:O", title="", sort=['Enero','Febrero','Marzo','Abril','Mayo','Junio','Julio','Agosto','Septiembre','Octubre','Noviembre','Diciembre']),
    y=alt.Y("Sales:Q", title='Ventas en Millones de Dólares'),
    tooltip=[alt.Tooltip('Sales')],
    color=alt.Color("NameYear:N")
).transform_window(
    rank="rank()",
    groupby=["Month"]
).properties(
    title="Ventas obtenidas por Mes",
    width=500,
    height=250,
)

alt.Chart(...)

1. Los datos contienen únicamente las **ventas obtendidas en el último mes del año 2017**.
2. El año 2021 tiene solamente registra **ventas en los meses de Enero, Febrero, Junio y Septiembre**.
3. Los años **2018, 2019 y 2020** tienen unas ventas más **uniformes** que los años 2017 y 2021.

#### Análisis Multivariante

In [18]:
corr_table = mrk_data.corr()
corr_table = corr_table.reset_index()
corr_table = corr_table.melt('index')
corr_table.columns = ['variable1','variable2','correlacion']
corr_table['corrRedondeado'] = corr_table.apply(lambda x: round(x['correlacion'], 2), axis=1 )

In [19]:
base = alt.Chart(corr_table).mark_rect().encode(
    x=alt.X('variable1:O',title='',axis=alt.Axis(labelFontSize=12, labelFontWeight='bold')),
    y=alt.Y('variable2:O',title='',axis=alt.Axis(labelFontSize=12, labelFontWeight='bold')),
    color=alt.Color('correlacion:Q', scale=alt.Scale(scheme='purples')),
    tooltip=['variable1','variable2','correlacion']
).properties(
    width=300,
    height=300
)

In [20]:
base = alt.Chart(corr_table).mark_rect().encode(
    x=alt.X('variable1:O',title='',axis=alt.Axis(labelFontSize=12, labelFontWeight='bold')),
    y=alt.Y('variable2:O',title='',axis=alt.Axis(labelFontSize=12, labelFontWeight='bold')),
    color=alt.Color('correlacion:Q', scale=alt.Scale(scheme='purples')),
    tooltip=['variable1','variable2','correlacion']
).properties(
    width=300,
    height=300
)

In [21]:
text = base.mark_text().encode(
    text='corrRedondeado',
    color=alt.condition(
        alt.datum.corrRedondeado > 0.8, 
        alt.value('white'),
        alt.value('black')
    )
)

In [22]:
base + text

alt.LayerChart(...)

→ El presupuesto de publicidad televisiva está **fuertemente relaciando** al de publicidad radiofónica.

#### Estadísticas Descriptivas

In [23]:
mrk_data.describe().T

,count,mean,std,min,25%,50%,75%,max
TV,655.0,376.569466,70.095769,168.000000,330.000000,381.000000,426.000000,582.000000
Radio,655.0,126.651154,25.916728,54.290910,109.738682,127.398413,145.510209,213.556056
Social Media,655.0,23.171277,5.787302,4.740290,19.029863,23.026443,27.066847,44.799380
Sales,655.0,1341.683214,248.858195,590.336016,1178.471164,1357.921625,1518.550852,2061.018042


## ML

In [24]:
'''
1. Antes de pasar los presupuestos deberíamos escalarlos? resuelta
2. Por qué se transforma primero con el carryover y luego el saturation?
3. Igualmente debería considerar el one hot encoding?
'''

'\n1. Antes de pasar los presupuestos deberíamos escalarlos? resuelta\n2. Por qué se transforma primero con el carryover y luego el saturation?\n3. Igualmente debería considerar el one hot encoding?\n'

In [25]:
'''class ExponentialSaturation: # Neto de la curva
    def __init__(self, a = 1.0):
        self.a = a
    def transform (self, X):
        return 1 - np.exp (-self.a * X) #→ 1 - e^(-a*x)'''

'class ExponentialSaturation: # Neto de la curva\n    def __init__(self, a = 1.0):\n        self.a = a\n    def transform (self, X):\n        return 1 - np.exp (-self.a * X) #→ 1 - e^(-a*x)'

In [26]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_is_fitted, check_array

class ExponentialSaturation (BaseEstimator, TransformerMixin):
    def __init__ (self, a = 1.):
        self.a = a
    def fit(self, X, y = None):
        X = check_array(X) 
        self._check_n_features(X, reset = True) # de BaseEstimator
    def transform (self, X):
        check_is_fitted (self)
        X = check_array (X)
        self._check_n_features (X, reset = False) # de BaseEstimator (para qué es el reset?)
        return 1 - np.exp (-self.a * X)

In [27]:
from scipy.signal import convolve2d

class ExponentialCarryover (BaseEstimator, TransformerMixin): 
    def __init__(self, strength = 0.5, length = 1):
        self.strength = strength
        self.length = length
        
    def fit(self, X, y = None):
        X = check_array(X)
        self._check_n_features(X, reset=True)
        self.sliding_window_ = (# no sé que hace esta línea
            self.strength ** np.arange(self.length + 1)
        ).reshape(-1, 1)#¿Cómo es un reshape -1,1?
        return self

    def transform(self, X: np.ndarray):
        check_is_fitted(self)
        X = check_array(X)
        self._check_n_features(X, reset=False)
        convolution = convolve2d(X, self.sliding_window_) # No entiendo esta línea
        if self.length > 0:# Por qué?
            convolution = convolution[: -self.length]
        return convolution

In [28]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


# Por ajustar 
adstock = ColumnTransformer(
    [
        ('tv_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
        ]),['TV']),
        ('radio_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
        ]), ['Radio']),
        ('banners_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
        ]), ['Banners']),
    ]
    )
model = Pipeline([
        ('adstock', adstock),
        ('regression', LinearRegression())
    ])